# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,rikitea,-23.1203,-134.9692,24.58,84,68,8.58,PF,1671536284
1,1,mattru,7.6244,-11.8332,32.86,46,64,0.97,SL,1671536284
2,2,souillac,-20.5167,57.5167,29.21,58,20,6.17,MU,1671536285
3,3,dalvik,65.9702,-18.5286,-1.04,92,100,17.11,IS,1671536285
4,4,ushuaia,-54.8000,-68.3000,9.81,76,0,3.09,AR,1671535908


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    alpha = 0.5
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] > 15) & (city_data_df["Max Temp"] < 20) & (city_data_df["Cloudiness"] > 50) & (city_data_df["Wind Speed"] > 2.5)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,ribeira grande,38.5167,-28.7000,18.22,94,75,12.07,PT,1671536291
42,42,lata,40.1629,-8.3327,17.26,89,80,4.37,PT,1671536305
82,82,lebu,-37.6167,-73.6500,15.12,71,99,6.04,CL,1671536328
155,155,castro,-24.7911,-50.0119,18.93,81,95,2.57,BR,1671536365
161,161,codrington,-38.2667,141.9667,15.79,61,94,6.47,AU,1671536368
242,242,launceston,-41.4500,147.1667,15.80,72,100,4.12,AU,1671536411
250,250,kruisfontein,-34.0033,24.7314,19.38,69,95,6.72,ZA,1671536415
253,253,port lincoln,-34.7333,135.8667,19.82,73,61,2.90,AU,1671536363
347,347,marsa matruh,31.3525,27.2453,17.06,68,100,8.42,EG,1671536464
360,360,nishihara,26.1842,127.7558,19.60,66,75,4.63,JP,1671536472


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,ribeira grande,PT,38.5167,-28.7000,94,
42,lata,PT,40.1629,-8.3327,89,
82,lebu,CL,-37.6167,-73.6500,71,
155,castro,BR,-24.7911,-50.0119,81,
161,codrington,AU,-38.2667,141.9667,61,
242,launceston,AU,-41.4500,147.1667,72,
250,kruisfontein,ZA,-34.0033,24.7314,69,
253,port lincoln,AU,-34.7333,135.8667,73,
347,marsa matruh,EG,31.3525,27.2453,68,
360,nishihara,JP,26.1842,127.7558,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
categories = "accommodation.hotel"

params = {
    "apiKey":geoapify_key,
    "limit":limit,
    "categories":categories
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira grande - nearest hotel: Hospedaria JSF - 3 Estrelas
lata - nearest hotel: Residencial Botânico
lebu - nearest hotel: No hotel found
castro - nearest hotel: Hotel Central Palace
codrington - nearest hotel: No hotel found
launceston - nearest hotel: Hotel Charles
kruisfontein - nearest hotel: No hotel found
port lincoln - nearest hotel: Boston Hotel
marsa matruh - nearest hotel: صقور
nishihara - nearest hotel: ユインチホテル南城
mount gambier - nearest hotel: The Old Mount Gambier Gaol
todos santos - nearest hotel: Hotel Casa Tota
santona - nearest hotel: Hotel Juan de la Cosa
gibraleon - nearest hotel: No hotel found
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
lagoa - nearest hotel: Casa da Madrinha
pisco - nearest hotel: La Portada


,City,Country,Lat,Lng,Humidity,Hotel Name
16,ribeira grande,PT,38.5167,-28.7000,94,Hospedaria JSF - 3 Estrelas
42,lata,PT,40.1629,-8.3327,89,Residencial Botânico
82,lebu,CL,-37.6167,-73.6500,71,No hotel found
155,castro,BR,-24.7911,-50.0119,81,Hotel Central Palace
161,codrington,AU,-38.2667,141.9667,61,No hotel found
242,launceston,AU,-41.4500,147.1667,72,Hotel Charles
250,kruisfontein,ZA,-34.0033,24.7314,69,No hotel found
253,port lincoln,AU,-34.7333,135.8667,73,Boston Hotel
347,marsa matruh,EG,31.3525,27.2453,68,صقور
360,nishihara,JP,26.1842,127.7558,66,ユインチホテル南城


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot

hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 2,
    alpha = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)